# Explore Different Solar Sizing 

In [6]:
from src.CAPEX import *
import math
from typing import Tuple
import plotly.express as px


In [ ]:
# Function to give better representation of solar and inverter price based on economy of scale
def solar_or_inverter_cost(size_MW: float,
                           price_1: float = 0.29,     # $/W at 1 MW
                           price_100: float = 0.22,   # $/W at 100 MW
                           price_min: float = 0.21    # asymptotic floor (market floor)
                          ) -> Tuple[float, float]:
    """
    Return (price_per_watt, total_cost_usd) for module/inverter cost at size_MW (MW_DC).
    Uses a saturating power-law with a floor so discounts flatten at utility scale.
    """

    # Ensure anchors are above the floor
    price_1 = max(price_1, price_min + 1e-6)
    price_100 = max(price_100, price_min + 1e-6)

    # Solve exponent so p(1)=p1 and p(100)=p100, then evaluate p(size_MW)
    b = math.log((price_100 - price_min) / (price_1 - price_min)) / math.log(100.0)
    price_per_w = price_min + (price_1 - price_min) * (size_MW ** b)

    total_cost_usd = price_per_w * size_MW * 1_000_000.0

    return price_per_w, total_cost_usd


(0.29, 290000.0)

In [ ]:
# MW sizes we want to test
MW_DC = [*range(1, 21), *range(25, 100, 5), *range(100, 255, 10)]

# Variable to hold all resulting dataframes
results = []
for size in MW_DC:
    panel_cost_per_watt = solar_or_inverter_cost(size, price_1=0.29, price_100=0.22, price_min=0.21)[0]
    inverter_cost_per_watt = solar_or_inverter_cost(size, price_1=0.0424, price_100=0.031, price_min=0.029)[0]
    results.append(get_CAPEX(project_size_MW_DC=size,
                             module_price_watt_DC=panel_cost_per_watt,
                             inverter_price_watt_DC=inverter_cost_per_watt,
                             sales_tax_percent=5,
                             substation_upgrade_YES_or_NO="NO"))

In [ ]:
# Per MW result for each category
per_MW = pd.concat([df["Cost $/W_dc"].rename(size) for size, df in zip(MW_DC, results)],axis=1).T
per_MW.index.name = "Size MW"
per_MW.reset_index().rename(columns={"index": "Size MW"})

# Total cost for each category
cost_MW = pd.concat([df["Total Cost"].rename(size) for size, df in zip(MW_DC, results)],axis=1).T
cost_MW.index.name = "Size MW"
cost_MW.reset_index().rename(columns={"index": "Size MW"})


CAPEX,Racking,Overhead and Profit,Install Labor,Solar Panels,General Construction,Gen-Tie,Asset Management,AC EBOS,Inverter,Anchoring and Mooring,DC EBOS,Engineering,Substation,Project Cost
Size MW,,,,,,,,,,,,,,
1,4.290533e+05,3.531756e+05,2.900592e+05,2.900000e+05,1.533728e+05,1.005807e+05,9.224852e+04,8.603550e+04,4.240000e+04,4.015385e+04,3.952663e+04,8.579882e+03,0.0,1.925186e+06
2,4.107692e+05,4.729631e+05,5.439053e+05,5.370015e+05,2.711243e+05,1.033617e+05,1.421302e+05,8.733728e+04,7.812789e+04,7.798817e+04,8.994083e+04,2.331361e+04,0.0,2.837963e+06
3,6.161538e+05,6.344526e+05,7.979290e+05,7.761401e+05,3.113609e+05,1.064387e+05,1.904734e+05,8.786982e+04,1.125363e+05,1.135740e+05,1.469822e+05,3.923077e+04,0.0,3.933142e+06
4,8.215385e+05,8.063602e+05,1.054438e+06,1.011117e+06,3.491124e+05,1.112907e+05,2.400000e+05,1.746746e+05,1.462337e+05,1.502485e+05,2.111243e+05,5.538462e+04,0.0,5.131522e+06
5,1.026923e+06,9.544649e+05,1.310059e+06,1.243394e+06,3.825444e+05,1.148410e+05,2.908284e+05,1.763314e+05,1.794647e+05,1.889941e+05,2.798817e+05,7.455621e+04,0.0,6.222283e+06
6,1.232308e+06,1.095537e+06,1.567101e+06,1.473733e+06,4.136095e+05,1.156694e+05,3.394083e+05,1.757396e+05,2.123575e+05,2.259408e+05,3.504142e+05,9.443787e+04,0.0,7.296255e+06
7,1.437692e+06,1.247348e+06,1.817101e+06,1.702589e+06,4.465089e+05,1.232915e+05,3.876923e+05,2.634320e+05,2.449900e+05,2.627692e+05,4.171006e+05,1.159763e+05,0.0,8.466490e+06
8,1.643077e+06,1.374224e+06,2.057751e+06,1.930262e+06,4.724260e+05,1.279069e+05,4.369231e+05,2.631953e+05,2.774135e+05,3.003077e+05,4.738462e+05,1.353846e+05,0.0,9.492717e+06
9,1.848462e+06,1.500002e+06,2.300592e+06,2.156962e+06,4.957988e+05,1.294454e+05,4.867456e+05,2.652071e+05,3.096642e+05,3.378462e+05,5.378698e+05,1.592308e+05,0.0,1.052783e+07


In [ ]:
fig = px.line(per_MW*1_000_000)
fig.update_layout(yaxis_title="Cost ($/MW)")
fig.show()

fig = px.line(cost_MW)
fig.update_layout(yaxis_title="Total Cost ($)")
fig.show()

In [87]:
from src.Solar import getSolarGen

zip_code = 83251

solar_1_MW_10_deg = getSolarGen(zip_code,
                                size=1, 
                                dc_ac_ratio=1.2, 
                                efficiency=86, 
                                tilt=10, 
                                num_years=1)

solar_1_MW_15_deg = getSolarGen(zip_code,
                                size=1, 
                                dc_ac_ratio=1.2, 
                                efficiency=86, 
                                tilt=15, 
                                num_years=1)

df = pd.DataFrame({"MW_DC": MW_DC})
df = df.set_index("MW_DC")
df["Tilt 10"] = 0
df["Tilt 15"] = 0
df

for size in MW_DC:
    df.loc[size, f"Tilt 10"] = solar_1_MW_10_deg.sum() * size
    df.loc[size, f"Tilt 15"] = solar_1_MW_15_deg.sum() * size

/var/folders/ff/qq2xmzm570q1wrmdvtm2mldm0000gn/T/ipykernel_69391/1819109060.py:26: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1710.1591866666668' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/ff/qq2xmzm570q1wrmdvtm2mldm0000gn/T/ipykernel_69391/1819109060.py:27: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1779.6881799999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [96]:
solar = solar_1_MW_15_deg.to_frame()

solar = solar.rename(columns={"Solar Generation (MW)": "Tile 15"})
solar["Tilt 10"] = solar_1_MW_10_deg.values

fig = px.line(solar)
fig.update_layout(yaxis_title="Generation (MW)")
fig.show()

In [97]:
fig = px.line(df)
fig.update_layout(yaxis_title="Annual Generation (MWh)")
fig.show()
        